In [2]:
import torch
import torchvision
import numpy as np
from PIL import Image
from tqdm import tqdm
from functools import partial

import pandas as pd
import random
from os import path
import os

In [3]:
ROOT_DIR = path.join('/home', 'victor', 'datasets')

DT_ROOT = path.join(ROOT_DIR, 'VOC')
DT_DEST_BINARY = path.join(ROOT_DIR, 'VOC_FORMS')
DT_DEST_RGB_RANDOM = path.join(ROOT_DIR , 'VOC_FORMS_RGB')
DT_DEST_RGB_SINGLE_CLASS = lambda c: path.join(ROOT_DIR, f'VOC_FORMS_RGB_{c.upper()}')

object_categories = ['aeroplane', 'bicycle', 'bird', 'boat',
                     'bottle', 'bus', 'car', 'cat', 'chair',
                     'cow', 'diningtable', 'dog', 'horse',
                     'motorbike', 'person', 'pottedplant',
                     'sheep', 'sofa', 'train', 'tvmonitor']

In [4]:
def gen_example_from_voc(voc):
    for example, segmentation in voc:
        im = Image.fromarray(np.array(segmentation) > 0).resize((512, 512)).convert("1")
        area = np.sum(im)
        yield (im, area)

def random_class_mask_generator(voc, im_shape=(224, 224)):
    for example, segmentation in voc:
        example, segmentation = example.resize(im_shape), segmentation.resize(im_shape)
        present_labels = np.setdiff1d(np.unique(segmentation), [0, 255])
        chosen = np.random.choice(present_labels)
        background = Image.fromarray((np.asarray(segmentation) != chosen).astype(np.bool))
        example.paste(0, mask=background)
        area = np.logical_not(background).sum()
        yield (example, area)

def class_mask_generator(voc, cl, im_shape=(224, 224)):
    for example, segmentation in voc:
        example, segmentation = example.resize(im_shape), segmentation.resize(im_shape)
        present_labels = np.setdiff1d(np.unique(segmentation), [0, 255])
        background = Image.fromarray((np.asarray(segmentation) != cl).astype(np.bool))
        area = np.logical_not(background).sum()
        if area == 0: continue
        
        example.paste(0, mask=background)
        yield (example, area)

def gen_df_from_voc(root_dir, dt, generator_fn, skip=True):
    root_dir = path.abspath(root_dir)
    img_dir = path.join(root_dir, "images")
    df_dest = path.join(root_dir, "data.csv")
    if skip and path.exists(df_dest):
        print(f"Found existing dataset, skipping for {root_dir}...")
        return pd.read_csv(df_dest, index_col=0)

    for directory in [root_dir, img_dir]:
        if not path.exists(directory):
            os.makedirs(directory)
            print(f"Created directory {directory}")

    df = pd.DataFrame(columns=["filename", "label"])
    for i, (img, area) in enumerate(generator_fn(dt)):
        filename = f"img_{i}.jpeg"
        dest_path = path.join(img_dir, filename)
        img.save(dest_path)
        row = pd.Series({"filename": filename, "label": area})
        df.loc[i] = row

    df.to_csv(df_dest)
    return df

In [5]:
def main():
    dt_train = torchvision.datasets.VOCSegmentation(
        root=path.join(DT_ROOT, 'train'),
        download=False,
        image_set='train'
    )

    dt_val = torchvision.datasets.VOCSegmentation(
        root=path.join(DT_ROOT, 'test'),
        download=False,
        image_set='val'
    )
    
    ## Binary ##
    gen_df_from_voc(
        path.join(DT_DEST_BINARY, "train"), dt_train, generator_fn=gen_example_from_voc
    )
    gen_df_from_voc(path.join(DT_DEST_BINARY, "test"), dt_val, generator_fn=gen_example_from_voc)

    ## RGB RANDOM CLASS ##
    gen_df_from_voc(
        path.join(DT_DEST_RGB_RANDOM, "train"),
        dt_train,
        generator_fn=random_class_mask_generator,
    )
    gen_df_from_voc(
        path.join(DT_DEST_RGB_RANDOM, "test"), dt_val, generator_fn=random_class_mask_generator
    )


    ## RGB SINGLE CLASS ##
    for cl in tqdm(object_categories):
        generator_fn = partial(class_mask_generator, cl=1 + object_categories.index(cl))
        gen_df_from_voc(
            path.join(DT_DEST_RGB_SINGLE_CLASS(cl), 'train'),
            dt_train,
            generator_fn=generator_fn
        )
        gen_df_from_voc(
            path.join(DT_DEST_RGB_SINGLE_CLASS(cl), 'test'),
            dt_val,
            generator_fn=generator_fn
        )

Found existing dataset, skipping for /home/victor/datasets/VOC_FORMS/train...
Found existing dataset, skipping for /home/victor/datasets/VOC_FORMS/test...
Created directory /home/victor/datasets/VOC_FORMS_RGB/train
Created directory /home/victor/datasets/VOC_FORMS_RGB/train/images
Created directory /home/victor/datasets/VOC_FORMS_RGB/test
Created directory /home/victor/datasets/VOC_FORMS_RGB/test/images


  0%|          | 0/20 [00:00<?, ?it/s]

Created directory /home/victor/datasets/VOC_FORMS_RGB_AEROPLANE/train
Created directory /home/victor/datasets/VOC_FORMS_RGB_AEROPLANE/train/images
Created directory /home/victor/datasets/VOC_FORMS_RGB_AEROPLANE/test
Created directory /home/victor/datasets/VOC_FORMS_RGB_AEROPLANE/test/images


  5%|▌         | 1/20 [00:11<03:34, 11.30s/it]

Created directory /home/victor/datasets/VOC_FORMS_RGB_BICYCLE/train
Created directory /home/victor/datasets/VOC_FORMS_RGB_BICYCLE/train/images
Created directory /home/victor/datasets/VOC_FORMS_RGB_BICYCLE/test
Created directory /home/victor/datasets/VOC_FORMS_RGB_BICYCLE/test/images


...

 90%|█████████ | 18/20 [03:26<00:23, 11.54s/it]

Created directory /home/victor/datasets/VOC_FORMS_RGB_TRAIN/train
Created directory /home/victor/datasets/VOC_FORMS_RGB_TRAIN/train/images
Created directory /home/victor/datasets/VOC_FORMS_RGB_TRAIN/test
Created directory /home/victor/datasets/VOC_FORMS_RGB_TRAIN/test/images


 95%|█████████▌| 19/20 [03:37<00:11, 11.49s/it]

Created directory /home/victor/datasets/VOC_FORMS_RGB_TVMONITOR/train
Created directory /home/victor/datasets/VOC_FORMS_RGB_TVMONITOR/train/images
Created directory /home/victor/datasets/VOC_FORMS_RGB_TVMONITOR/test
Created directory /home/victor/datasets/VOC_FORMS_RGB_TVMONITOR/test/images


100%|██████████| 20/20 [03:48<00:00, 11.45s/it]


In [ ]:
if __name__ == "__main__": main()